In [1]:
import numpy as np
from tqdm import tqdm

DEFAULT_FILENAME = "input12.txt"
def read_lines(filename = DEFAULT_FILENAME):
    res = []
    with open(filename, "r") as f:
        for li in f:
            instr, arr = li.strip().split(' ')
            instr = np.array([*instr])
            arr = [int(a) for a in arr.split(',')]
            instr_np = np.zeros(len(instr), dtype=int)
            instr_np[np.where(instr == '#')] = 1
            instr_np[np.where(instr == '?')] = 2
            res.append((instr_np, arr))
    return res

In [2]:
def reduce_spaces(spaces):
  instructions = []
  prev_elem = 0
  for elem in spaces:
    if elem == 1 and prev_elem == 0:
      instructions.append(1)
    elif elem == 1 and prev_elem == 1:
      instructions[-1] += 1
    prev_elem = elem
  return instructions
def try_naive(line):
  spaces, instructions = line
  instructions_diff = np.sum(instructions) - np.where(spaces == 1)[0].size
  count = 0
  slots = np.where(spaces == 2)[0]
  for i in range(2 ** slots.size):
    for j in range(slots.size):
      spaces[slots[j]] = (i >> j) % 2
    if reduce_spaces(spaces) == instructions:
      count += 1
  return count

values = read_lines()
count_total = 0
for line in tqdm(values):
  count_total += try_naive(line)
print(f"Part 1: {count_total}")

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:38<00:00, 26.07it/s]

Part 1: 7922


In [3]:
def try_smart(spaces, instructions):
  if len(instructions) == 0:
    return not np.any(spaces == 1)
  # pick the middle and start going
  ins_ind = len(instructions) // 2
  pad_before = int(np.sum(instructions[:ins_ind])) + len(instructions[:ins_ind])
  pad_after = int(np.sum(instructions[ins_ind+1:])) + len(instructions[ins_ind+1:])
  res = 0
  for i in range(pad_before, spaces.size - pad_after - instructions[ins_ind]+1):
    if np.all(spaces[i:i + instructions[ins_ind]] > 0):
      if i == 0 or spaces[i-1] != 1:
        if i + instructions[ins_ind] == len(spaces) or spaces[i + instructions[ins_ind]] != 1:
          left = try_smart(spaces[:i - (0 if i == 0 else 1)], instructions[:ins_ind])
          if left > 0:
            right = try_smart(spaces[i+instructions[ins_ind]+(0 if i + instructions[ins_ind] == len(spaces) else 1):], instructions[ins_ind+1:])
            res += left * right
  return res


values = read_lines()
count_total = 0
for line in tqdm(values):
  count_total += try_smart(line[0], line[1])
print(f"Part 1: {count_total}")
count_total = 0
for line in tqdm(values):
  count_total += try_smart(np.concatenate([line[0], [2], line[0], [2], line[0], [2], line[0], [2], line[0]]), line[1] * 5)
print(f"Part 2: {count_total}")

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 4050.69it/s]


Part 1: 7922


100%|██████████| 1000/1000 [05:24<00:00,  3.08it/s]

Part 2: 18093821750095
